<a href="https://colab.research.google.com/github/KRTSZ/blog/blob/master/building_a_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Analytics Edge of Retail Optimization

Both binary variables and non-linear functions


#gurobipy
The Gurobi Optimizer is a mathematical optimization software library for solving mixed-integer linear and quadratic optimization problems.

In [4]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 103.6 MB/s eta 0:00:00


# load require packages

In [5]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import plotly.express as px

### Lists

In [6]:
# Be careful with Python indices!
# The range function range(a,b) creates a range of integers starting at <a> but ending at <b-1>
weeks = list(range(157,170))
print("weeks (t) ", weeks)
# Also note here that the first element of a list has index 0. The second element has index 1, and so on...
w1 = weeks[0] #denotes the number of the first week in our planning horizon
print(w1)
w2 = weeks[1] #denotes the number of the second week in our planning horizon
print(w2)

weeks (t)  [157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169]
157
158


### Parameters

In [7]:
#From orginal price / sales(demand) data
# Developed linear demand model: used 27-156 weeks (3 years) of data; 27-104( years 1/2)Train, 105-156( last year)Test

# Variable: 𝑆𝑒𝑎𝑠𝑜𝑛X. Year divided int 13 seasons / 4 weeks per season (categorical variable)
# Variable: Price,  at time 𝑝_𝑡, 𝑝_(𝑡−1), 𝑝_(𝑡−2)
# Linear regression developed demand model (Constrain 1)
# Demand is characterized by our linearly additive model
# model: 𝑑_𝑡 = 1.978242858 − 2.809634145*𝑝_𝑡 + 0.963410728*𝑝_(𝑡−1) + 0.759639170*𝑝_(𝑡−2) −0.562046910*𝑆𝑒𝑎𝑠𝑜𝑛2 +...+  0.947945548*𝑆𝑒𝑎𝑠𝑜𝑛13

# Store model parameters
intercept = 1.978242858
# causal factors
p_coeff = -2.809634145
p1_coeff = 0.963410728
p2_coeff = 0.759639170
# time series factor
season_coeff = {1: 0, 2: -0.562046910, 3: 0.087545274,  4: -0.402637480, 5: -0.027326010, 6: 0.004349885,
               7: -0.036102297, 8: -0.069280527, 9: 0.160276197, 10: 1.104208897, 11: 1.122711287, 12: 1.176802194, 13: 0.947945548}

In [8]:
# Here we create a dictionary that associates a season with each week in the planning horizon
season = {}
for w in weeks:
    season[w] = np.ceil((w % 52) / 4)

print(season)

{157: np.float64(1.0), 158: np.float64(1.0), 159: np.float64(1.0), 160: np.float64(1.0), 161: np.float64(2.0), 162: np.float64(2.0), 163: np.float64(2.0), 164: np.float64(2.0), 165: np.float64(3.0), 166: np.float64(3.0), 167: np.float64(3.0), 168: np.float64(3.0), 169: np.float64(4.0)}


In [9]:
p_ladder = [1.0, 0.95, 0.85, 0.75, 0.60, 0.50] #create a list for price ladder

In [10]:
# Create model object
mod2 = gp.Model ("price_model_2")

Restricted license - for non-production use only - expires 2026-11-23


In [11]:
# Define decision variables which includes ladder constraint
p = mod2.addVars(weeks)
x = mod2.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [12]:
# Set objective function
obj_fn = mod2.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [13]:
# Select price ladder value
constr_select_ladder = mod2.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod2.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)

In [14]:
mod2.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 26 rows, 91 columns and 169 nonzeros
Model fingerprint: 0xbaf8ae07
Model has 36 quadratic objective terms
Variable types: 13 continuous, 78 integer (78 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 8.4722613
Presolve removed 0 rows and 13 columns
Presolve time: 0.00s
Presolved: 26 rows, 78 columns, 143 nonzeros
Presolved model has 36 quadratic objective terms
Variable types: 13 continuous, 65 integer (65 binary)

Root relaxation: objective 1.046838e+01, 40 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Object

In [15]:
df2 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df2.loc[w,"price"] = p[w].x
df2

,price
157,0.95
158,0.95
159,0.85
160,0.85
161,0.75
162,0.75
163,0.75
164,0.75
165,0.85
166,0.85


In [16]:
fig = px.line(df2, x=df2.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

### End of script
---
